In [1]:
import pandas as pd

In [ ]:
applied_data = pd.read_json("./train_data/Applied_data.json")
aptitude_data_1 = pd.read_json("./train_data/Aptitude.json")
aptitude_data_2 = pd.read_json("./train_data/Aptitude_2.json")
interview_1 = pd.read_json('./train_data/Interview.json')
interview_2 = pd.read_json('./train_data/Interview_Next_Stage_2.json')
oa_data = pd.read_json('./train_data/OA.json')
offer_data_1 = pd.read_json('./train_data/Offer.json')
offer_data_2 = pd.read_json('./train_data/Offer_2.json')
rejected = pd.read_json('./train_data/Rejected.json')

applied_data['label'] = applied_data.apply(lambda x: 0,axis =1)
oa_data['label'] = oa_data.apply(lambda x: 1,axis =1)
aptitude_data_1['label'] = aptitude_data_1.apply(lambda x: 1,axis =1)
aptitude_data_2['label'] = aptitude_data_2.apply(lambda x: 1,axis =1)
interview_1['label'] = interview_1.apply(lambda x: 2,axis =1)
interview_2['label'] = interview_2.apply(lambda x: 2,axis =1)
offer_data_1['label'] = offer_data_1.apply(lambda x: 3,axis =1)
offer_data_2['label'] = offer_data_2.apply(lambda x: 3,axis =1)
rejected['label'] = rejected.apply(lambda x: 4,axis =1)

df_jobs = pd.concat([applied_data, aptitude_data_1, aptitude_data_2, interview_1, interview_2, oa_data, offer_data_1, offer_data_2, rejected])
def remove_n(row):
    text = row['body']
    text = text.replace('\n' ," ")
    return text
df_jobs['body'] = df_jobs.apply(lambda x: remove_n(x),axis=1)
final_df = df_jobs.sample(frac=1).reset_index(drop=True)
def append_text(row):
    subj = row['subject']
    body = row['body']

    text = "Subject: " + subj + " Body:" + body 
    return text

final_df['text'] = final_df.apply(lambda x: append_text(x),axis=1)
final_df.to_csv("./train_data/job_stage_data.csv",index=False)


In [1]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="./train_data/email_classification_data.csv")

In [2]:
# Label 0 --> Shortlisted for interview
# Label 1 --> Rejected
# Label 2 --> Documents
# Label 3 --> Offer


dataset['train'].to_pandas()

,sentence,label
0,"Dear [Candidate's Name],\n\nI hope this email ...",0
1,"Dear Emily,\n\nI hope this email finds you wel...",0
2,hope this email finds you well. I wanted to p...,0
3,"Dear Emily,\n\nI hope this email finds you wel...",0
4,hope this message finds you in good health. I'...,0
5,Hope you're doing well! 😊 I've got some great ...,0
6,"Dear [Candidate's Name],\n\nI trust this messa...",0
7,"Dear [Candidate's Name],\n\nCongratulations on...",0
8,"Hi Akshit,\n \nIt was nice speaking to you ear...",0
9,I hope this message finds you well. I wanted t...,1


In [3]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset



# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset['train'],
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=4,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"sentence": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1440
  Num epochs = 1
  Total optimization steps = 360
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

In [4]:
trainer.model._save_pretrained("job_stage_classification_model")

In [6]:
# Download from Hub and run inference
model = SetFitModel.from_pretrained("job_stage_classification_model")
# Run inference


In [8]:
def infer_one_sentence(sentence):
        predtext = [sentence]
        predicted_class = model(predtext)
        return str(predicted_class)

In [9]:
infer_one_sentence("You have been shortlisted for the interview")

'tensor([0], dtype=torch.int32)'

In [10]:
infer_one_sentence("We appreciate your interest in SAP and the time you’ve invested in applying for the Software Developer Intern (SAP AI) opening. Unfortunately we will not be moving forward with your application, nevertheless we would like to share additional information with you, so please take a minute to watch this video message.")

'tensor([1], dtype=torch.int32)'

In [11]:
infer_one_sentence("please send your NRIC and passport to move on further.")

'tensor([2], dtype=torch.int32)'

In [12]:
infer_one_sentence("""thank you for returning my call earlier! J

As shared, we are happy to offer you an internship opportunity with IHiS for the project: NLP modelling & visualisation on clinical textual data

Kindly refer to the details below for the internship details.

Allowance:                  $1800 per month
Period:                        9 Jan 2023 – 26 May 2023 (credit bearing internship)
Office hours:               830am to 6pm (Monday to Thursday) and 830am to 530pm (Friday), 1h lunch break
Location                      1 Maritime Square #12-01 Harbourfront Centre Singapore 099253""")

'tensor([3], dtype=torch.int32)'